In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730611475.621724      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1103 05:24:35.636243103      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1103 05:24:35.636258412      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1103 05:24:35.636261654      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1103 05:24:35.636263920      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1103 05:24:35.636266190      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730611491.861856      13 service.cc:145] XLA service 0x56faf171bed0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730611491.861920      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730611491.861925      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730611491.861928      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730611491.861932      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730611491.861934      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730611491.861937      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730611491.861940      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730611491.861943      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 92
    config_dict = {'csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
    config_dict['parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
else:
    num_jobs = 4
    config_dict = {'csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'}
    config_dict['parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

print(len(final_id_list))
print(len(train_data_id_list))

996
3960


In [7]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [8]:
# Calculate the approximate size of each smaller list
chunk_size = len(train_data_id_list) // num_jobs
remainder = len(train_data_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(train_data_id_list[start:end])
    start = end

# Check the result
"""for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")"""

'for i, sublist in enumerate(result):\n    print(f"List {i + 1} (length {len(sublist)}): {sublist}")'

In [9]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [10]:
def transform_parquet_file(parquet_file_path, file_id):

    directory = Path(parquet_file_path)

    # Check if any Parquet file exists in the directory
    parquet_files = list(directory.glob("*.parquet"))

    if parquet_files:
        parquet_df = pl.read_parquet(parquet_file_path+"*.parquet")
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(file_id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
    else:
        columns = [("id", pl.String), ("weekday", pl.Int8), ("hour_of_day", pl.Int32), ("relative_date_PCIAT", pl.Float32)\
                  , ("enmo_avg", pl.Float32), ("anglez_avg", pl.Float32), ("light_avg", pl.Float32)\
                   , ("battery_voltage_avg", pl.Float32)]

        parquet_df = pl.DataFrame(schema=columns)
    
    return parquet_df

In [11]:
def unify_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()
    parquet_file_name = "id={0}/".format(df_initializing_id)
    first_parquet_file = config_dict['parquet_root_path']+parquet_file_name
    first_parquet_df = transform_parquet_file(first_parquet_file, df_initializing_id)
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)
    first_joined_features_df = train_data_of_id.join(first_parquet_df, on="id", how="left")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/".format(id)
        parquet_file_path = config_dict['parquet_root_path']+parquet_file_name
        parquet_df = transform_parquet_file(parquet_file_path, id)
        
        train_data_of_id = train_data.filter(pl.col('id')==id)
        joined_features = train_data_of_id.join(parquet_df, on="id", how="left")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [12]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_files)(id_list) for id_list in result))

100%|██████████| 42/42 [00:59<00:00,  1.41s/it]


In [34]:
test_df = parts_of_parquet[0]

test_df = test_df.drop(['PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total', 'sii'])

#dd.sql("select * from test_df limit 100").pl()

In [35]:
from sklearn.preprocessing import StandardScaler

test_df_pandas = test_df.to_pandas()
#print(test_df_pandas["Physical-BMI"].isna().any())
complete_column = "id"

missing_mask = test_df_pandas.drop(columns=[complete_column]).isna()

for col in missing_mask.columns:
    test_df_pandas[col] = test_df_pandas[col].fillna(test_df_pandas[col].mean())

#test_df_pandas.head(100)

In [36]:
import pandas as pd
scaler = StandardScaler()

df_scaled_values = scaler.fit_transform(test_df_pandas.drop(columns=[complete_column]))
df_scaled = pd.DataFrame(df_scaled_values, columns=test_df_pandas.drop(columns=[complete_column]).columns)

# Add the complete column back to the scaled DataFrame
df_scaled[complete_column] = test_df_pandas[complete_column].values
df_scaled = df_scaled[test_df_pandas.columns]  # Ensure original column order

#df_scaled.head(100)

In [37]:
# Parameters for the MICE-like imputation
iterations = 17  # Number of iterations for imputation
epochs = 35  # Training epochs for each imputer model
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync  # Batch size for training


# Function to create a simple neural network model for imputation
def create_imputer_model(input_dim):
    df_input = tf.keras.layers.Input(shape=(input_dim,), name="df_input")
    hidden_layer1 = tf.keras.layers.Dense(85, activation="relu")(df_input)
    hidden_layer2 = tf.keras.layers.Dense(65, activation="relu")(hidden_layer1)
    hidden_layer3 = tf.keras.layers.Dense(35, activation="relu")(hidden_layer2)
    df_output = tf.keras.layers.Dense(1)(hidden_layer3)
    
    model = tf.keras.Model(df_input, df_output)
    model.compile(optimizer="adamax", loss="mean_squared_error")
    return model

In [17]:
import numpy as np

col = 'PAQ_A-PAQ_A_Total'
missing_rows = missing_mask[col]

if not missing_rows.any():
    print("no missing data; skipping over")
else:
    X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
    y_train = df_scaled.loc[~missing_rows, col].values
    X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    
    pred_ds = tf.data.Dataset.from_tensor_slices(X_pred)
    pred_ds = pred_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
    pred_ds = pred_ds.prefetch(tf.data.AUTOTUNE)

In [33]:
test_df.columns

#PAQ_A-PAQ_A_Total, PAQ_C-PAQ_C_Total, sii

#dd.sql('select distinct "PAQ_A-PAQ_A_Total" from train_data where "PAQ_A-PAQ_A_Total" is not null ').pl()

#dd.sql('select id, count(1) from train_data where "PAQ_C-PAQ_C_Total" is not null group by id order by 2 desc').pl()

['id',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-CGAS_Score',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Waist_Circumference',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSND',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_GSD',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'PAQ_A-PAQ_A_Total',
 'PAQ_C-PAQ_C_Total',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 '

In [27]:
with tpu_strategy.scope():
    imputer_model = create_imputer_model(input_dim=X_train.shape[1])
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    imputer_model.fit(train_ds, epochs=epochs, callbacks=[early_stopping_cb])

Epoch 1/35


2024-11-02 12:37:39.934388: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730551060.007023     923 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(ff3c6585fb93079b:0:0), session_name()


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3726
Epoch 2/35


I0000 00:00:1730551060.361463     923 tpu_compile_op_common.cc:245] Compilation of ff3c6585fb93079b:0:0 with session name  took 354.359194ms and succeeded
I0000 00:00:1730551060.365484     923 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(ff3c6585fb93079b:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_12586302780582042046", property.function_library_fingerprint = 8114739729407744959, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730551060.365516     923 tpu_compilation_cache_interface.cc:541] After adding entry for key ff3c6585fb93079b:0:

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1065 
Epoch 3/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0541 
Epoch 4/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0500 
Epoch 5/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0593 
Epoch 6/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0530 
Epoch 7/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0374 
Epoch 8/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0194 
Epoch 9/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0147  
Epoch 10/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0120 
Epoch 11/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0103 
Epoch 12/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0089 
Epoch 13/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0083 
Epoch 14/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0074 
Epoch 15/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0068 
Epoch 16/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0065 
Epoch 17/35
2

In [28]:
with tpu_strategy.scope():
    predicted_values = imputer_model.predict(pred_ds)

df_scaled.loc[missing_rows, col] = predicted_values

3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step 

I0000 00:00:1730551102.323606     921 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(46b927f6009dca76:0:0), session_name()
I0000 00:00:1730551102.381366     921 tpu_compile_op_common.cc:245] Compilation of 46b927f6009dca76:0:0 with session name  took 57.725163ms and succeeded
I0000 00:00:1730551102.381725     921 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(46b927f6009dca76:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_3001699512433967351", property.function_library_fingerprint = 13391374956198879495, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partit

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step


I0000 00:00:1730551102.743559     920 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(d23a5895cd15e6bf:0:0), session_name()
I0000 00:00:1730551102.814819     920 tpu_compile_op_common.cc:245] Compilation of d23a5895cd15e6bf:0:0 with session name  took 71.223258ms and succeeded
I0000 00:00:1730551102.815377     920 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d23a5895cd15e6bf:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_4749942726664743389", property.function_library_fingerprint = 13578819203757294200, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,87,;", property.guaranteed_constants_size = 0, embedding_

In [29]:
df_scaled[test_df_pandas.drop(columns=[complete_column]).columns] \
= scaler.inverse_transform(df_scaled[test_df_pandas.drop(columns=[complete_column]).columns])

df_imputed = df_scaled

In [30]:
df_imputed

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,16.0,0.0,61.940529,21.937701,66.500000,138.000000,31.000000,70.000000,99.000000,...,3.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
1,00008ff9,5.0,0.0,51.000000,16.877316,46.000000,50.800000,26.846154,63.538413,82.602194,...,4.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
2,000fd460,9.0,0.0,61.940529,14.035590,48.000000,46.000000,22.000000,75.000000,70.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3,00105258,10.0,1.0,71.000000,16.648696,56.500000,75.600000,26.846154,65.000000,94.000000,...,4.0,2.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
4,00115b9f,9.0,0.0,71.000000,18.292347,56.000000,81.600000,26.846154,60.000000,97.000000,...,4.0,3.0,2.0,6.000000,5.000000,43.000000,0.044082,-14.720180,14.921977,4139.472656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0219f64a,6.0,0.0,68.000000,16.377681,48.500000,54.800000,24.000000,67.000000,81.000000,...,4.0,4.0,1.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3200,021b2020,9.0,1.0,61.940529,20.631322,53.500000,84.000000,28.000000,64.000000,81.000000,...,4.0,3.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3201,0232fd98,14.0,1.0,61.940529,22.860295,59.866617,113.389711,26.846154,63.538413,82.602194,...,4.0,4.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3202,02347f13,10.0,0.0,61.940529,20.033783,59.000000,99.200000,28.000000,79.000000,88.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475


In [33]:
df_imputed["Physical-BMI"].isna().any()
#df_imputed.isna().any()

False

In [38]:
df_scaled.columns

Index(['id', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score',
       'Physical-BMI', 'Physical-Height', 'Physical-Weight',
       'Physical-Waist_Circumference', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
       'PCIAT-PCIAT_04', 'PCIAT-P

In [ ]:
with tpu_strategy.scope():
    for iteration in range(iterations):
        print(f"\nIteration {iteration + 1}/{iterations}")

        # Iterate over each column to impute, excluding the complete column
        for col in df_scaled.columns:
            print("Starting with column --> {0}".format(col))

            if col == complete_column:
                print("complete column; skipping over")
                continue  # Skip the complete column

            # Use the original missing mask to find rows with missing values
            missing_rows = missing_mask[col]

            if not missing_rows.any():
                print("no missing data; skipping over")
                continue
            
            X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
            y_train = df_scaled.loc[~missing_rows, col].values
            X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

            train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
            train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
            train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

            pred_ds = tf.data.Dataset.from_tensor_slices(X_pred)
            pred_ds = pred_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
            pred_ds = pred_ds.prefetch(tf.data.AUTOTUNE)

            # Create and train the imputer model
            imputer_model = create_imputer_model(input_dim=X_train.shape[1])
            early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
            imputer_model.fit(train_ds, epochs=epochs, callbacks=[early_stopping_cb], verbose=2)

            # Predict missing values and update the DataFrame
            predicted_values = imputer_model.predict(pred_ds)
            df_scaled.loc[missing_rows, col] = predicted_values
            
    df_scaled[test_df_pandas.drop(columns=[complete_column]).columns] \
    = scaler.inverse_transform(df_scaled[test_df_pandas.drop(columns=[complete_column]).columns])

    df_imputed = df_scaled


Iteration 1/17
Starting with column --> id
complete column; skipping over
Starting with column --> Basic_Demos-Age
no missing data; skipping over
Starting with column --> Basic_Demos-Sex
no missing data; skipping over
Starting with column --> CGAS-CGAS_Score


I0000 00:00:1730612865.726470      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1/35


2024-11-03 05:47:50.278624: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730612870.501284     916 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(fba23dcc0d06549f:0:0), session_name()
I0000 00:00:1730612870.881326     916 tpu_compile_op_common.cc:245] Compilation of fba23dcc0d06549f:0:0 with session name  took 379.990655ms and succeeded
I0000 00:00:1730612870.885856     916 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(fba23dcc0d06549f:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_12197948064361817849", property.function_library_fingerprint = 18232892938838354722, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z 

25/25 - 2s - 94ms/step - loss: 1.9978
Epoch 2/35


...

Starting with column --> Physical-BMI
Epoch 1/35


I0000 00:00:1730614811.944048     881 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(8b66c660ce1aac94:0:0), session_name()
I0000 00:00:1730614812.298041     881 tpu_compile_op_common.cc:245] Compilation of 8b66c660ce1aac94:0:0 with session name  took 353.958191ms and succeeded
I0000 00:00:1730614812.301466     881 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(8b66c660ce1aac94:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_16043735412279469204", property.function_library_fingerprint = 14932553364519910494, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "16,84,;16,;", property.guaranteed_constants_size = 0, embedding_p

21/21 - 2s - 107ms/step - loss: 1.0982
Epoch 2/35


I0000 00:00:1730614812.786738     924 tpu_compile_op_common.cc:245] Compilation of 235e06821498cc26:0:0 with session name  took 339.992531ms and succeeded
I0000 00:00:1730614812.789019     924 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(235e06821498cc26:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_16043735412279469204", property.function_library_fingerprint = 14932553364519910494, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "13,84,;13,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730614812.789050     924 tpu_compilation_cache_interface.cc:541] After adding entry for key 235e068

21/21 - 1s - 27ms/step - loss: 0.6218
Epoch 3/35
21/21 - 1s - 29ms/step - loss: 0.3135
Epoch 4/35
21/21 - 1s - 29ms/step - loss: 0.1390
Epoch 5/35
21/21 - 1s - 28ms/step - loss: 0.0502
Epoch 6/35
21/21 - 1s - 28ms/step - loss: 0.0393
Epoch 7/35
21/21 - 1s - 28ms/step - loss: 0.0342
Epoch 8/35
21/21 - 1s - 28ms/step - loss: 0.0293
